<a href="https://colab.research.google.com/github/robgon-art/GreenLIT/blob/main/GreenLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GreenLIT: Using GPT-J with Multi-Task Learning to Create New Screenplays**
## How to fine-tune an ML model to create TV shows and movies with new titles, plot summaries, and scripts

![ReGEN Cover Image](https://raw.githubusercontent.com/robgon-art/ReGEN/main/cover_med.jpg)

Photo by [Tech Daily](https://unsplash.com/photos/PGuCnUzsRSM) on [Unsplash](https://unsplash.com/)</br>

**By Robert. A Gonsalves**</br>
You can see my article on [Medium](https://towardsdatascience.com/greenlit-using-gpt-j-with-multi-task-learning-to-create-new-screenplays-54a2d04f761c#c07d-fe51a662351d).

In [ ]:
#@title Download the Pretrained Model
!pip install -U --no-cache-dir gdown --pre
!gdown 103xsaUZlukpbOu-h2AEyAyQHwMHwgRbk

In [ ]:
#@title Initialize the System
!nvidia-smi
# !pip install transformers bitsandbytes-cuda111 wikipedia
!pip install transformers==4.22.2
!pip install bitsandbytes==0.34.0
!pip install wikipedia

from torch import nn
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from torch.cuda.amp import custom_fwd, custom_bwd
import torch.nn.functional as F
import wikipedia
import transformers
import torch

import nltk
nltk.download('wordnet')

config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

def check_in_wiki(name):
  name_parts = name.split()
  wiki_results = wikipedia.search(name)
  for w in wiki_results:
    w = w.lower()
    match_all_parts = True
    for n in name_parts:
      n = n.lower()
      if n == "the" or n == "a":
        continue
      if n not in w:
        match_all_parts = False
        break
    if match_all_parts:
      return True
  return False

class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias

class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

gpt = torch.load("/content/GreenLIT_new.pt",  map_location=torch.device('cuda'))
gpt.eval()

import re
import textwrap
from nltk.corpus import wordnet as wn

In [6]:
#@title Generate Titles and Summaries
genre = 'crime drama' #@param {type:"string"}
theme = 'cryptocurrency' #@param {type:"string"}

prompt = "GENRE: " + genre + " THEME: " + theme + "TITLE:"
with torch.no_grad():
  prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  sample_outputs = gpt.generate(prompt_tokens, max_length=80, do_sample=True, 
    temperature=0.8, pad_token_id=tokenizer.eos_token_id, num_return_sequences=40)

titles = []
summaries = []
count = 1
for i, sample_output in enumerate(sample_outputs):
  results = tokenizer.decode(sample_output, skip_special_tokens=True)
  results = results.replace("\n", " ")
  genre = re.search('GENRE:(.*)THEME:', results).group(1).strip()
  title = re.search('TITLE:(.*)SUMMARY:', results).group(1).strip()

  already_done = check_in_wiki(title)
  alpha = re.sub('[^a-zA-Z]+', '', title)

  if len(alpha) < 3 or already_done:
    continue

  summary = re.search('SUMMARY:(.*)', results).group(1).strip()
  titles.append(title)
  summaries.append(summary)

  out = str(count).zfill(2) + " " + title + " - " + summary
  wrapped = textwrap.fill(out, width=150, subsequent_indent="   ")
  print(wrapped)
  count += 1

01 L'Inconnu de l'Amérique - In this suspenseful mystery about an unknown man found dead in a New York building, the detective and the doctor
   investigate a large number of clues to discover who the man really was and why he died.
02 The Cryptovalley - In search of financial independence, a young man discovers that cryptocurrency is a way to unlock a world of opportunities. But
   when he buys into the promise of a promising industry, he becomes a target for everyone who wants to take advantage of the newcomers.
03 The Dark Side of the Internet - In this crime thriller, a young man uses the anonymous currency of Bitcoin to help kidnap his ex-girlfriend.
04 Unforeseen (4K UHD) - Unforeseen is a darkly comedic drama about an unlikely friendship as five men with nothing in common but a common obsession,
   Bitcoin, go on the run from a pair of armed robbers.
05 The Private Life of Pippa Lee - A young woman, trapped in a loveless marriage, is drawn into exploring her husband's world of c

In [7]:
#@title Choose a Title and Create a Script

choice = 2 #@param {type:"slider", min:1, max:40, step:1}
choice -= 1

if choice >= len(titles):
  print("Choose between 1 and " + str(len(titles)))
else:
  title = titles[choice]
  summary = summaries[choice]
  print(choice+1, title)

  prompt = "TITLE: " + title + " SUMMARY: " + summary + " SCRIPT:\n[Scene:"
  
  with torch.no_grad():
    prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    sample_outputs = gpt.generate(prompt_tokens, max_length=480, do_sample=True, 
      top_k=50, pad_token_id=tokenizer.eos_token_id,
      num_return_sequences=1)
    results = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    results = results.strip()
    print("\n[Scene:" + results[len(prompt):])

2 The Cryptovalley

[Scene: Club; the club is packed. Ozzy is in his element.]

OSBORNE - And this is his new friend, the Cryptovalley guy, Timmy.
TIMMYS - Hi Ozzy. I'm Timmy.
OSBORNE - Welcome to our humble abode, folks. We're having a special night in honour of our new friend. As you may have heard, the Cryptovalley is now known as the hottest thing since the Aztec empire. A few of our customers got together and put together a deal that really could change the world. Here ya go.
[He pulls out a velvet bag.
TIMMYS - That's...
OSBORNE - Let me handle this, Timothy. As some of you know, I came in on this just before the explosion. And I made a little bit of money. But Timmy was actually thinking of entering the new game.
TIMMYS - Yeah. Sure.
OSBORNE - But he didn't need to. Because as you can see Timmy came to us for a certain...
[Enter Anson. He walks up to Ozzy, looks at Timmy and his expression changes. He gives Ozzy a look of absolute disgust. Ozzy's expression changes to that of co